In [ ]:
default = ['.ipynb_checkpoints', 'cudart32_75.dll', 'cudart64_75.dll', 'FDTD_2D_Full.exe', 'FDTD_TEST.ipynb', 'Settings.ini']
import re
import os
import subprocess
import numpy as np
import shutil
import copy
"""
Функция формирует массив-лист, содержащий первым элементом имя параметра "name",
а далее - подряд n копий массива "arr"
"""
def setArr1(name, arr, n):
    cArr = arr.copy()
    for i in range(n - 1):
        cArr = np.concatenate((cArr, arr))
    cArr = cArr.tolist()
    cArr.insert(0, name)
    return cArr
"""
Функция формирует массив-лист, содержащий первым элементом имя параметра "name",
а далее - подряд n копий массива "arr", значения в которых увеличены на j * step,
где j - порядковый номер копии
"""
def setArr2(name, arr, step, n):
    cArr = arr.copy()
    for i in range(n - 1):
        cArr = np.concatenate((cArr, [j + (i + 1) * step for j in arr]))
    cArr = cArr.tolist()
    cArr.insert(0, name)
    return cArr

def G(x, freq1, freq2, freq_step):
    freqArr = np.arange(freq1, freq2, freq_step)
    expCount = len(freqArr)
    tau_0 = [9417.526568, 9677.774038, 9998.078616, 10302.177308, 10695.885018]
    #print(tau_0)
    
    SET = [setArr2('FlucIcen', x[0] * np.ones(expCount, int), 0, 1),
           setArr2('FlucJcen', x[1] * np.ones(expCount, int), 0, 1),
           setArr2('FlucA', x[2] * np.ones(expCount, int), 0, 1),
           setArr2('FlucB', x[3] * np.ones(expCount, int), 0, 1),
           setArr2('FlucC', x[4] * np.ones(expCount, int), 0, 1),
           setArr2('FlucAngleI', x[5] * np.ones(expCount, int), 0, 1),
           setArr2('FlucAngleJ', x[6] * np.ones(expCount, int), 0, 1),
           setArr2('FlucAngleK', x[7] * np.ones(expCount, int), 0, 1),
           setArr2('FlucNe0', x[8] * np.ones(expCount, int), 0, 1),
           setArr1('FreqA', freqArr, 1)]
    #массив-сценарий
    
    #for i in range(len(SET)):
        #print("\n", SET[i])
    a = open('Settings.ini') #открытие настроечного файла
    b = a.readlines() #чтение настроечного файла по строкам
    g = 0
    f = False
    for exp in range(expCount):
        #print('\n------\tExperiment №' + str(exp + 1), "\n")
        for i in SET:
            for j in range(len(b)):
                if re.match(i[0], b[j]):
                    d = b[j]
                    q = d.find('=')
                    for k in range(len(i[0]), q):
                        if d[k].isdigit() or d[k].isalpha():
                            f = True
                            break
                    if f == True:
                        f = False
                        continue
                    #print('old:\t' + b[j])
                    for k in range(len(d) - q):
                        if (d[q + k].isdigit()) or (d[q + k] == '-'):
                            q1 = q + k
                            break
                    for k in range(len(d) - q1):
                        if not(d[q1 + k].isdigit()) and not(d[q1 + k] == '.') and not(d[q1 + k] == '-'):
                            q2 = q1 + k
                            break
                    d = d[:q1] + str(i[exp + 1]) + d[q2:]
                    #print('new:\t' + d)
                    b[j] = d
                    a = open('Settings.ini', 'w')
                    a.writelines(b)
                    a.close()
        runexp = subprocess.call('FDTD_2D_Full')
        #if runexp == 0:
            #print('Experiment №' + str(exp + 1) + ' done.\n')
        a = np.loadtxt('det_ARC.txt', usecols = (1, 3))
        g += abs(a[200][1] - tau_0[3])
        print('tau_0 = ' + str(tau_0[3]) + ', tau = ' + str(a[200][1]))
        ff = False
        for m in os.listdir():
            for n in default:
                if m == n:
                    ff = True
            if ff == False:
                os.remove(m)
            ff = False
    return g

In [ ]:
"""
Метод исследующего поиска
"""
def m1(i, j, step_ij, maxSteps, eps, dcrsStep, inrcsStep):
    x_curr = [i, j, 1.5, 1.5, 1.5, 180, 180, 180, 0.5] #начальные координаты флуктуации
    step = [(2 * step_ij) / maxSteps, (2 * step_ij) / maxSteps, 3 / maxSteps, 3 / maxSteps, 3 / maxSteps, 360 / maxSteps, 360 / maxSteps, 360 / maxSteps] #массив размера шага по каждый из координат
    initPoint = copy.deepcopy(x_curr)
    eps_break = 0 #счётчик шагов в пределах погрешности
    k = 0 #порядковый номер изменяемой координаты
    goodSteps = 0 #счётчик успешных шагов
    steps = 0 #счётчик всех шагов
    mem = False #для запоминания первого значения G_0
    G_init = 0 #значение G_0 в начальной точке
    exp = 0 #счётчик проведённых экспериментов
    fail= False #флаг неверного предыдущего шага
    lastGood = False
    goodStepsList = [];
    print('--------------------------------\nBEGIN at: i = ' + str(i) + ', j = ' + str(j))
    print('step values: \t' + str(step))
    x_tmp = copy.deepcopy(x_curr) #координаты для следующего шага
    x_tmp[k] += step[k] 
    print('current point: \t' + str(x_curr) + ',\nnext point: \t' + str(x_tmp))
    while 1:
        if fail == False: #не рассчитываем G_0, если предыдущий шаг по этой же координате был неверным (избегаем повторного расчёта)
            if lastGood == True:
                G_0 = G_1
            else:
                G_0 = G(x_curr, 56, 58, 2) #расчёт G в текущей точке
                exp += 1
                lastGood = False
            if mem == False: #запомнить значение G_0 в начальной точке
                G_init = G_0
                mem = True
        G_1 = G(x_tmp, 56, 58, 2) #расчёт G в точке следующего шага
        exp += 1
        G_dif =  G_0 - G_1 #расчёт разницы G между текущей точкой и точкой следующего шага
        print('G_0 = ' + str(G_0) + ', G_1 = ' + str(G_1) + ', G_dif = ' + str(G_dif) + ', exp = ' + str(exp))
        if abs(G_dif) < eps: #разница меньше допустимой погрешности
            eps_break += 1
            print('\nEPS-WARNING')
        else:
            eps_break = 0
        if eps_break == 14: #если шаги по всем координатам в пределах допустимой погрешности, закончить поиск
            print('\nEPS-BREAK')
            break
        if G_dif < 0: #разница меньше нуля - верный шаг
            print('\nGOOD step')
            goodStepsList.insert(0, x_tmp)
            lastGood = True
            fail = False
            step[k] *= dcrsStep
            print('step values: \t' + str(step))
            k += 1
            goodSteps += 1
            if k > 7:
                k = 0
                steps += 1
            print('steps: \t' + str(steps) + ', good steps: \t' + str(goodSteps))
            if steps == maxSteps: #если количество шагов больше максимального, закончить поиск
                print('\nMAX STEPS BREAK')
                break
            x_curr = copy.deepcopy(x_tmp)
            x_tmp[k] += step[k]
            print('\nNEXT STEP:\ncurrent point: \t' + str(x_curr) + ',\nnext point: \t' + str(x_tmp))
        else:
            print('\nBAD step')
            if fail == True:
                print('JUMP to the next step')
                step[k] *= inrcsStep
                if steps == maxSteps:
                    print('\nMAX STEPS BREAK')
                    break
                if step[k] > initPoint[k] / (maxSteps - steps):
                    step[k] = initPoint[k] / (maxSteps - steps)
                fail = False
                k += 1
                if k > 7:
                    k = 0
                    steps += 1
                print('steps: \t' + str(steps) + ', good steps: \t' + str(goodSteps))
                x_tmp = copy.deepcopy(x_curr)
                x_tmp[k] += step[k]
                print('step values: \t' + str(step))
                print('\nNEXT STEP:\ncurrent point: \t' + str(x_curr) + ',\nnext point: \t' + str(x_tmp))
                continue
            print('TURN around')
            fail = True
            step[k] *= -1
            x_tmp = copy.deepcopy(x_curr)
            x_tmp[k] += step[k]
            print('step values: \t' + str(step))
            print('steps: \t' + str(steps) + ', good steps: \t' + str(goodSteps))
            print('current point: \t' + str(x_curr) + ',\nnext point: \t' + str(x_tmp))
    if G_0 > G_1:
        print('END: G_init: ' + str(G_init) + ', G_end: ' + str(G_0) + ', ' + str(x_curr))
        print('good steps: ' + str(goodSteps) + ', whole steps: ' + str(steps))
        goodStepsList
        return G_0
    else:
        print('END: G_init: ' + str(G_init) + ', G_end: ' + str(G_1) + ', ' + str(x_tmp))
        print('good steps: ' + str(goodSteps) + ', whole rounds: ' + str(steps))
        goodStepsList
        return G_1

In [4]:
m1(200, 1200, 200, 50, 1e-2, 1.2, 0.8)
m1(200, 1300, 200, 50, 1e-2, 1.2, 0.8)

--------------------------------
BEGIN at: i = 200, j = 1200
step values: 	[8.0, 8.0, 0.06, 0.06, 0.06, 7.2, 7.2, 7.2]
current point: 	[200, 1200, 1.5, 1.5, 1.5, 180, 180, 180, 0.5],
next point: 	[208.0, 1200, 1.5, 1.5, 1.5, 180, 180, 180, 0.5]
tau_0 = 9417.526568, tau = 10597.696412
tau_0 = 9417.526568, tau = 10672.052832
G_0 = 1815.0592739999975, G_1 = 2038.1285339999977, G_dif = -223.0692600000002, exp = 2

GOOD step
step values: 	[9.6, 8.0, 0.06, 0.06, 0.06, 7.2, 7.2, 7.2]
steps: 	0, good steps: 	1

NEXT STEP:
current point: 	[208.0, 1200, 1.5, 1.5, 1.5, 180, 180, 180, 0.5],
next point: 	[208.0, 1208.0, 1.5, 1.5, 1.5, 180, 180, 180, 0.5]
tau_0 = 9417.526568, tau = 10913.711197
G_0 = 2038.1285339999977, G_1 = 2980.929808000001, G_dif = -942.8012740000031, exp = 3

GOOD step
step values: 	[9.6, 9.6, 0.06, 0.06, 0.06, 7.2, 7.2, 7.2]
steps: 	0, good steps: 	2

NEXT STEP:
current point: 	[208.0, 1208.0, 1.5, 1.5, 1.5, 180, 180, 180, 0.5],
next point: 	[208.0, 1208.0, 1.56, 1.5, 1.5, 180

929.9318910000002